# Week 05 – Web Security: Wapiti Vulnerability Scanning

This workshop demonstrates how to install, run, and interpret results from Wapiti — a web application vulnerability scanner.


In [4]:
# Install Wapiti
%pip install --upgrade pip
%pip install wapiti3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Verify installation
!wapiti --version


     __      __               .__  __  .__________
    /  \    /  \_____  ______ |__|/  |_|__\_____  \
    \   \/\/   /\__  \ \____ \|  \   __\  | _(__  <
     \        /  / __ \|  |_> >  ||  | |  |/       \
      \__/\  /  (____  /   __/|__||__| |__/______  /
           \/        \/|__|                      \/
Wapiti 3.2.3 (wapiti-scanner.github.io)
[*] You are lucky! Full moon tonight.
3.2.3


## Choose a Target
Set your own Google Gruyere instance URL here.


In [6]:
target_url = "https://google-gruyere.appspot.com/557573274340656254581601813589935313418/"  
out_name = "wapiti_report"

In [7]:
# Run Wapiti scan
cmd = f"wapiti -u {target_url} -o {out_name} -f html"
print("Running:", cmd)
!{cmd}

Running: wapiti -u https://google-gruyere.appspot.com/557573274340656254581601813589935313418/ -o wapiti_report -f html

     __      __               .__  __  .__________
    /  \    /  \_____  ______ |__|/  |_|__\_____  \
    \   \/\/   /\__  \ \____ \|  \   __\  | _(__  <
     \        /  / __ \|  |_> >  ||  | |  |/       \
      \__/\  /  (____  /   __/|__||__| |__/______  /
           \/        \/|__|                      \/
Wapiti 3.2.3 (wapiti-scanner.github.io)
Traceback (most recent call last):
  File "/opt/homebrew/bin/wapiti", line 7, in <module>
    sys.exit(wapiti_asyncio_wrapper())
  File "/opt/homebrew/lib/python3.10/site-packages/wapitiCore/main/wapiti.py", line 505, in wapiti_asyncio_wrapper
    asyncio.run(wapiti_main())
  File "/opt/homebrew/Cellar/python@3.10/3.10.19_1/Frameworks/Python.framework/Versions/3.10/lib/python3.10/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "/opt/homebrew/Cellar/python@3.10/3.10.19_1/Frameworks/Pyt

In [8]:
# Helper script to load the HTML report
import os, glob
from IPython.display import HTML, display

def find_html_report(out):
    if os.path.isfile(out) and out.endswith('.html'):
        return os.path.abspath(out)
    if os.path.isdir(out):
        files = glob.glob(os.path.join(out, '*.html'))
        if files:
            files.sort(key=os.path.getmtime, reverse=True)
            return files[0]
    files = glob.glob(out + '*.html') + glob.glob(out + '*/*.html')
    if files:
        files.sort(key=os.path.getmtime, reverse=True)
        return files[0]
    return None

report = find_html_report(out_name)
if report:
    print("Report found:", report)
    display(HTML(open(report, 'r', encoding='utf-8').read()))
else:
    print("No report found.")

No report found.


# Full Lab Report: Wapiti Vulnerability Assessment

## 1. Introduction
The purpose of this assessment was to evaluate the security posture of a deliberately vulnerable web application using the Wapiti web vulnerability scanner. Google Gruyere was chosen as the target due to its safe and controlled testing environment for learning offensive security concepts. The goal of this lab was to identify potential vulnerabilities, understand their implications, and provide mitigation strategies aligned with industry best practices and OWASP standards.

## 2. Methodology
### 2.1 Tools and Setup
- **Scanner:** Wapiti 3.2.3
- **Scan Type:** Black-box
- **Target:** https://google-gruyere.appspot.com/<instance>/
- **Output Format:** HTML and console report

### 2.2 Procedure
1. A unique Gruyere training instance was deployed.
2. The scanner was executed with:
   ```
   wapiti -u <target> -o wapiti_report -f html
   ```
3. Wapiti crawled 10 pages and injected payloads for multiple vulnerability classes.
4. Results were analysed and interpreted against OWASP testing guidelines.

### 2.3 Exclusions
- No authentication brute force
- No destructive payloads
- No manual exploitation

---

## 3. Findings

### 3.1 Summary Table

| Vulnerability Type | Count | Severity |
|--------------------|-------|----------|
| Reflected XSS | 1 | High |
| Missing CSP | 1 | Medium |
| Missing Clickjacking Protection | 1 | Medium |
| Missing HSTS | 1 | Medium |
| Missing X-Content-Type-Options | 1 | Medium |
| Cookie Attribute Weaknesses | 2 | Medium |

---

## 3.2 Reflected Cross-Site Scripting (High)
**Location:** `/snippets.gtl`  
**Parameter:** `uid`

### Description
User input is reflected directly in the response HTML without sanitisation or encoding. This enables malicious JavaScript execution in the victim’s browser.

### Impact
- Session hijacking  
- Account takeover  
- Cookie theft  
- Arbitrary JavaScript execution  
- Phishing and redirection  

### Mitigation
- Validate user input through whitelisting  
- Encode output before injecting into HTML templates  
- Implement a Content Security Policy (CSP)  
- Remove inline JavaScript patterns  

---

## 3.3 Missing Content-Security-Policy (Medium)
A CSP header can block unauthorised scripts and mitigate XSS.

### Mitigation
```
Content-Security-Policy: default-src 'self';
```

---

## 3.4 Missing Clickjacking Protection (Medium)
The absence of `X-Frame-Options` allows the application to be framed, enabling clickjacking attacks.

### Mitigation
```
X-Frame-Options: DENY
```
or
```
Content-Security-Policy: frame-ancestors 'none';
```

---

## 3.5 Missing HSTS (Medium)
Without HSTS, the application is exposed to SSL stripping and downgrade attacks.

### Mitigation
```
Strict-Transport-Security: max-age=63072000; includeSubDomains; preload
```

---

## 3.6 Missing MIME Sniffing Protection (Medium)
Missing the `X-Content-Type-Options` header allows browsers to guess MIME types, increasing XSS risk.

### Mitigation
```
X-Content-Type-Options: nosniff
```

---

## 3.7 Cookie Security Weaknesses (Medium)
`HttpOnly` and `Secure` flags are missing in the `GRUYERE` cookie.

### Mitigation
```
Set-Cookie: GRUYERE=value; HttpOnly; Secure;
```

---

## 4. Conclusion
The Wapiti assessment identified one critical vulnerability (Reflected XSS) and several medium-severity security misconfigurations. These results demonstrate the importance of secure development practices such as input validation, output encoding, and the use of modern security headers. Although Gruyere is intentionally vulnerable, this lab exercise demonstrates practical vulnerability discovery and remediation strategies.

